In [5]:
#Try running this cell if Jupyter Notebook is acting funny
!pip install ipywidgets
from IPython import display
display.clear_output()
print("Done")

Done


In [6]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU.")


CUDA is not available. Using CPU.


In [7]:
import os
HOME =  os.getcwd()
print(HOME)

c:\Users\user\Documents\GitHub\Oyster_Detection\S25_codebase


In [2]:
#Make the necessary imports
!pip install ultralytics==8.2.103 -q
!pip install roboflow

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i9-13900K)
Setup complete  (32 CPUs, 31.7 GB RAM, 561.2/930.6 GB disk)


In [8]:
#Import YOLO
from ultralytics import YOLO

from IPython.display import display, Image
print("Finished importing display, Image ✅")

Finished importing display, Image ✅


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="exampleaipkeyhere") #add your own api key. Free to get from Roboflow.com

project = rf.workspace("oyster-classification").project("reu_oyster_2024_improved-e5wxs")
version = project.version(2)
dataset = version.download("yolov8-obb")
from IPython import display
display.clear_output()
print("\nDone extracting files! ✅")

loading Roboflow workspace...
loading Roboflow project...


RuntimeError: {'error': 'yolov11-obb is an invalid format for project type object-detection. Please use one of: clip, coco, coco-mmdetection, createml, darknet, multiclass, tensorflow, tfrecord, voc, yolokeras, yolov4pytorch, yolov4scaled, yolov5-obb, yolov5pytorch, yolov7pytorch, yolov8, yolov8-obb, yolov9, yolov11, yolov12, mt-yolov6, retinanet, benchmarker, paligemma, paligemma-txt, florence2-od, openai.'}

In [8]:
#train many smaller models:
#this was trained on a RTX 4080.
#I trained one model at a time over the course of several hours, you may train multiple
#in a row if you have the time and watch a computer for a long time
!PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128 python OysterResearch.ipynb
import torch
from ultralytics import YOLO
print(f"Allocated Memory: {torch.cuda.memory_allocated() / 1024 / 1024:.2f} MB")

!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:32
models = ["yolov8m.pt"] #"yolov8n.pt", 
#image size is 960 for everything except medium, for gpu mem alloc issues
for m in models:
    model = YOLO(m)
    model.train(
        data=dataset.location + "/data.yaml",
        epochs=200,
        imgsz=640,
        batch=16,
        patience=20,
        amp=True,
        optimize=True, #amp=True, cache=False
        project="oyster_logs",
        name=f"model_{m.split('.')[0]}",
        exist_ok=True
    )


Traceback (most recent call last):
  File "/mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/OysterResearch.ipynb", line 181, in <module>
    "execution_count": null,
NameError: name 'null' is not defined
Allocated Memory: 5000.24 MB
New https://pypi.org/project/ultralytics/8.3.115 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/data.yaml, epochs=200, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=oyster_logs, name=model_yolov8m, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, 

train: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/train/labels.cache... 7916 images, 2415 backgrounds, 127 corrupt: 100%|██████████| 7916/7916 [00:00<?, ?it/s]

train: WARNING ⚠️ /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/train/images/02d690e1d96d45e09858b46e75c9ca49_jpg.rf.27f5e0499787429b624843bb8752d2ca.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0618]
train: WARNING ⚠️ /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/train/images/02d690e1d96d45e09858b46e75c9ca49_jpg.rf.7375414da8dee3cf7b983ef524864cff.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1041]
train: WARNING ⚠️ /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/train/images/02d690e1d96d45e09858b46e75c9ca49_jpg.rf.73a3e4db6201888a72febf8f30393c06.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0618]
train: WARNING ⚠️ /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/train/images/02d690e1d96d45e09858b46e75c9ca49_jpg.rf.a441fccf03


val: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/valid/labels.cache... 282 images, 249 backgrounds, 0 corrupt: 100%|██████████| 282/282 [00:00<?, ?it/s]


Plotting labels to oyster_logs/model_yolov8m/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to oyster_logs/model_yolov8m
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    116/200      12.5G     0.6433      0.462      1.053        111        640: 100%|██████████| 487/487 [01:20<00:00,  6.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:00<00:00,  9.80it/s]

                   all        282        482      0.908      0.843        0.9      0.699


EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 96, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

116 epochs completed in 2.719 hours.
Optimizer stripped from oyster_logs/model_yolov8m/weights/last.pt, 52.1MB
Optimizer stripped from oyster_logs/model_yolov8m/weights/best.pt, 52.1MB

Validating oyster_logs/model_yolov8m/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  7.45it/s]


                   all        282        482      0.865      0.871      0.901      0.702
 Oyster- Indeterminate         27        351      0.895      0.923      0.941      0.719
         Oyster-Closed         22         37      0.795      0.838      0.849      0.668
           Oyster-Open         29         94      0.906      0.851      0.914      0.719
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to oyster_logs/model_yolov8m


In [16]:
from ultralytics import YOLO
import pandas as pd
import os


##Validate Your Model(s) on testing data

# List of model paths (update with yours)
model_paths = [
    "oyster_logs/model_yolov8m/weights/best.pt",
    "oyster_logs/model_yolov8s/weights/best.pt",
    "oyster_logs/model_yolov8n/weights/best.pt"
]

# Location of your data.yaml file
# Swap as needed, full path as was just copied/pasted from file explorer
data_yaml = "/mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/data.yaml"

results_list = []
count = 0
names = ["YOLOv8m","YOLOv8s","YOLOv8n"]
for path in model_paths:
    
    print(f"Validating model: {path}")
    model = YOLO(path)
    results = model.val(data=data_yaml, save=False)

    results_list.append({
        "Run Name": names[count],
        "Precision": round(results.box.p.mean().item(), 3),
        "Recall": round(results.box.r.mean().item(), 3),
        "mAP@0.5": round(results.box.map50.mean().item(), 3),
        "mAP@0.5:0.95": round(results.box.map.mean().item(), 3)
    })
    count+=1




Validating model: oyster_logs/model_yolov8m/weights/best.pt
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/valid/labels.cache... 282 images, 249 backgrounds, 0 corrupt: 100%|██████████| 282/282 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00, 10.11it/s]


                   all        282        482      0.865      0.871      0.902      0.701
 Oyster- Indeterminate         27        351      0.895      0.924      0.941      0.718
         Oyster-Closed         22         37      0.795      0.838       0.85      0.674
           Oyster-Open         29         94      0.905      0.851      0.914      0.712
Speed: 0.3ms preprocess, 4.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val18
Validating model: oyster_logs/model_yolov8s/weights/best.pt
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/valid/labels.cache... 282 images, 249 backgrounds, 0 corrupt: 100%|██████████| 282/282 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  8.64it/s]


                   all        282        482       0.86      0.863      0.899      0.675
 Oyster- Indeterminate         27        351       0.92      0.895      0.929      0.706
         Oyster-Closed         22         37      0.743      0.811       0.84      0.598
           Oyster-Open         29         94      0.915      0.883       0.93       0.72
Speed: 0.7ms preprocess, 4.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val19
Validating model: oyster_logs/model_yolov8n/weights/best.pt
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/valid/labels.cache... 282 images, 249 backgrounds, 0 corrupt: 100%|██████████| 282/282 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00, 13.58it/s]


                   all        282        482      0.898       0.78      0.894      0.663
 Oyster- Indeterminate         27        351      0.879      0.843      0.925      0.663
         Oyster-Closed         22         37      0.867      0.706       0.85      0.653
           Oyster-Open         29         94      0.949      0.791      0.909      0.674
Speed: 0.7ms preprocess, 1.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val20


In [18]:
# Create and display DataFrame to neatly display metrics for all 3 models
df = pd.DataFrame(results_list)
df_sorted = df.sort_values(by="mAP@0.5", ascending=False).reset_index(drop=True)
print("\n📊 Model Comparison:")
print(df_sorted)


📊 Model Comparison:
  Run Name  Precision  Recall  mAP@0.5  mAP@0.5:0.95
0  YOLOv8m      0.865   0.871    0.902         0.701
1  YOLOv8s      0.860   0.863    0.899         0.675
2  YOLOv8n      0.898   0.780    0.894         0.663


In [12]:
# Evaluate ALL Models in specified path. Commented out, but could be useful if 
# further research and computer resources allow you to train many/for a long time

# for path in model_paths:
#     print(f"Evaluating: {path}")
#     model = YOLO(path)
# [
#     # Validate the model
#     results = model.val(data=data_yaml, save=False, verbose=False)

#     # Collect results
#     results_list.append({
#         "Run Name": os.path.basename(os.path.dirname(path)),
#         "Precision": round(results.box.p.mean().item(), 3),
#         "Recall": round(results.box.r.mean().item(), 3),
#         "mAP@0.5": round(results.box.map50.mean().item(), 3),
#         "mAP@0.5:0.95": round(results.box.map.mean().item(), 3)
#     })

# # Show results as a DataFrame
# df = pd.DataFrame(results_list)
# print("\n📊 Model Comparison:")
# print(df)

Evaluating: oyster_logs/model_yolov8m/weights/best.pt
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/valid/labels.cache... 282 images, 249 backgrounds, 0 corrupt: 100%|██████████| 282/282 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00,  9.72it/s]


                   all        282        482      0.865      0.871      0.902      0.701
Speed: 0.3ms preprocess, 4.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val7
Evaluating: oyster_logs/model_yolov8s/weights/best.pt
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/valid/labels.cache... 282 images, 249 backgrounds, 0 corrupt: 100%|██████████| 282/282 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  8.15it/s]


                   all        282        482       0.86      0.863      0.899      0.675
Speed: 0.7ms preprocess, 4.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val8
Evaluating: oyster_logs/model_yolov8n/weights/best.pt
Ultralytics YOLOv8.2.103 🚀 Python-3.9.2 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model/REU_Oyster_2024_Improved-2/valid/labels.cache... 282 images, 249 backgrounds, 0 corrupt: 100%|██████████| 282/282 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00, 13.84it/s]


                   all        282        482      0.898       0.78      0.894      0.663
Speed: 0.7ms preprocess, 1.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val9

📊 Model Comparison:
  Run Name  Precision  Recall  mAP@0.5  mAP@0.5:0.95
0  weights      0.865   0.871    0.902         0.701
1  weights      0.860   0.863    0.899         0.675
2  weights      0.898   0.780    0.894         0.663
3  weights      0.865   0.871    0.902         0.701
4  weights      0.860   0.863    0.899         0.675
5  weights      0.898   0.780    0.894         0.663


In [9]:
results_file = "Oyster_Orientation_Model/oyster_logs/model_yolov8m/results.csv"
model = YOLO("runs/detect/trainX/weights/best.pt") #trainX should be replaced according to your runs/detect directory
results = model.predict("example_file.png")  # Used on my home directory. Turn example_file.png --> Your_file.png
#This allows you to test a specific file

# Show the image with bounding boxes
results[0].show()
print("done")

FileNotFoundError: [Errno 2] No such file or directory: 'runs\\detect\\trainX\\weights\\best.pt'

In [23]:
import yaml
import os

print(os.getcwd())
with open(dataset.location + "/data.yaml", "r") as f:
    data_yaml = yaml.safe_load(f)

print("Classes:", data_yaml['names'])

for class_id, class_name in enumerate(data_yaml['names']):
    print(f"Class {class_id} ({class_name}):")
    !grep -c " {class_id}" ~/Oyster_Orientation_Model/REU_Oyster_2024_Improved-1/test/labels/*txt
print("done")

/mnt/linuxlab/home/ncorcoran1/Oyster_Orientation_Model


NameError: name 'dataset' is not defined

In [24]:
#Future user will need to change their code or their directory structure
from ultralytics import YOLO

model = YOLO("~/Oyster_Orientation_Model/oyster_logs/model_yolov8m/weights/best.pt")  # or wherever your model is saved
print("Got Model")

metrics = model.val(data="REU_Oyster_2024_Improved-2/data.yaml", imgsz=960, batch=16)
print("Got Metrics")

# View the results
print(metrics)


FileNotFoundError: [Errno 2] No such file or directory: '~/Oyster_Orientation_Model/oyster_logs/model_yolov8m/weights/best.pt'

In [22]:
import yaml
import os
from glob import glob

# Define path to dataset YAML
yaml_path = os.path.join(dataset.location, "data.yaml")

# Load class names from YAML
with open(yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)

class_names = data_yaml['names']
num_classes = len(class_names)

# Set test label directory
label_dir = os.path.join(dataset.location, "test/labels")
label_files = glob(os.path.join(label_dir, "*.txt"))

# Count instances per class
class_counts = [0] * num_classes

for label_file in label_files:
    with open(label_file, "r") as f:
        for line in f:
            if line.strip():
                class_id = int(line.split()[0])
                if 0 <= class_id < num_classes:
                    class_counts[class_id] += 1

# Print results
print("📊 Class Distribution in Test Set:")
for class_id, class_name in class_names.items():
    print(f"Class {class_id} ({class_name}): {class_counts[int(class_id)]} samples")


NameError: name 'dataset' is not defined